In [81]:
import pandas as pd
from gensim.models import word2vec
import artm
from nltk.stem.porter import PorterStemmer
from nltk import word_tokenize
from nltk import stem
from nltk import WordNetLemmatizer
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
import xgboost
from sklearn.linear_model import LogisticRegression
import datetime
pd.options.display.max_colwidth=200
import scipy
import time

import telegram

def send_to_telegram(text):

    """Send appropriate links to telegram channel"""

    bot = telegram.Bot(token='379005601:AAH1rv3ESXLWTXbn14gnCxW52eeKc4qnw50')
    # chat_id = -1001111732295
    chat_id = 169719023
    bot.send_message(chat_id=chat_id, text=text)
    time.sleep(15)

def No_with_word(token_text):

    """Concating NO with words"""

    tmp=''
    for i,word in enumerate(token_text):
        if word==u'не':
            tmp+=("_".join(token_text[i:i+2]))
            tmp+= ' '
        else:
            if token_text[i-1]!=u'не':
                tmp+=word
                tmp+=' '
    return tmp


def wrk_words_wt_no(sent):

    """Making lemmatisation for Russian and English text"""

    words=word_tokenize(basic_cleaning2(sent).lower())
    lemmatizer = WordNetLemmatizer()
    try:
        arr=[]
        for i in range(len(words)):
            arr.append(morph.parse(words[i])[0].normal_form)
        arr2 = []
        for i in arr:
            arr2.append(lemmatizer.lemmatize(i, pos='v'))
        arr3 = []
        for i in arr2:
            arr3.append(lemmatizer.lemmatize(i, pos='n'))
        arr4 = []
        for i in arr3:
            arr4.append(lemmatizer.lemmatize(i, pos='a'))
        words1=[w for w in arr4 if w not in stop]
#         words1=[w for w in arr4 if w not in english_stops]
        words1=No_with_word(words1)
        return words1
    except TypeError:
        pass

def basic_cleaning2(string):

    """Cleaning text from numbers and punctuation"""

    string = string.lower()
    string = re.sub('[0-9\(\)\!\^\%\$\'\"\.;,-\?\{\}\[\]\\/]', ' ', string)
    string = re.sub(' +', ' ', string)
    return string





def make_bow(col):

    """Make bow from train_test"""

    binVectorizer = CountVectorizer(binary=True, ngram_range=(1, 1), min_df = 2, max_df=20000, max_features=4000)
#     binVectorizer = CountVectorizer(binary=True, ngram_range=(1, 1), min_df = 2)

    counts = binVectorizer.fit_transform(np.array(collection))
    bow_df = pd.DataFrame(counts.toarray(), columns=binVectorizer.get_feature_names())
    to_include = []
    for i in bow_df.columns:
        if bow_df[i].sum()>=2:
            to_include.append(i)
    bow_df = bow_df[to_include]
#     bow_df = scipy.sparse.csr_matrix(bow_df.values)
    return bow_df




def review_to_wordlist(review):

    """Convert collection to wordlist"""

    words = review.lower().split()
    words = [w for w in words]
    return(words)

def make_w2v(collection):

    """Make w2v model"""

    collection_splited = pd.DataFrame(collection).apply(lambda x:x.apply(review_to_wordlist))
    model = word2vec.Word2Vec(collection_splited[0], sg=1, size=300, window=10, workers=4)
    w2v = dict(zip(model.wv.index2word, model.wv.syn0))
    return w2v, model

In [72]:
word2vec.Word2Vec?

In [61]:
def make_w2v_mean(collection):

    """Mean transformation"""

    collection_df = pd.DataFrame(collection).apply(lambda x:x.apply(review_to_wordlist))
#     collection_df = pd.DataFrame(collection)
    data_mean=mean_vectorizer(w2v).fit(collection_df[0]).transform(collection_df[0])
    w2v_mean = pd.DataFrame(data_mean).reset_index(drop=True)
    return w2v_mean

def make_w2v_tfidf(collection):

    """Tf-idf transformation"""

    collection_df = pd.DataFrame(collection).apply(lambda x:x.apply(review_to_wordlist))
    data_tfidf=tfidf_vectorizer(w2v).fit(collection_df[0]).transform(collection_df[0])
    w2v_tfidf = pd.DataFrame(data_tfidf).reset_index(drop=True)
    return w2v_tfidf

class mean_vectorizer(object):

    """Preprocess word embedding using mean transformation"""

    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = len(next(iter(w2v.values())))

    def fit(self, X):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

class tfidf_vectorizer(object):

    """Class to preprocess word embedding using tf-idf"""

    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        self.dim = len(next(iter(w2v.values())))

    def fit(self, X):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

        return self

    def transform(self, X):
        return np.array([
                np.mean([self.word2vec[w] * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])

def make_artm(col):

    """Get artm theta matrixes"""

    collection_train = pd.DataFrame(collection).iloc[train_index].reset_index()
    collection_test = pd.DataFrame(collection).iloc[test_index].reset_index()
    le=LabelEncoder()
    y_transormed = le.fit_transform(df_y)

    arr=[]
    for index_number, i, c in zip(collection_train['index'], collection_train[0], y_transormed):
        arr.append(str(index_number) + ' |@default_class ' + str(i) + ' |@labels_class ' + str(c))

    arr_test=[]
    for index_number, i in zip(collection_test['index'], collection_test[0]):
        arr_test.append(str(index_number) +  ' |@default_class ' + str(i))

    pd.DataFrame(arr,index=None).to_csv('leaver_vw_form.txt',sep='\t',encoding='UTF-8',index=False, header=None)
    pd.DataFrame(arr_test,index=None).to_csv('leaver_vw_form_test.txt',sep='\t',encoding='UTF-8',index=False, header=None)

    batch_vectorizer = artm.BatchVectorizer(data_path="leaver_vw_form.txt", data_format="vowpal_wabbit", target_folder="leaver_vw_form_train", batch_size=100)
    batch_vectorizer_test = artm.BatchVectorizer(data_path="leaver_vw_form_test.txt", data_format="vowpal_wabbit", target_folder="leaver_vw_form_test", batch_size=100)

    T = pd.DataFrame(df_y)[target].nunique()
    print ("количество тем составляет - {}".format(T))# количество тем
    topic_names=["sbj"+str(i) for i in range(T)]

    model_artm = artm.ARTM(num_topics=T,
                           topic_names=topic_names,
                           class_ids={'@default_class':1, '@labels_class':700},
                           num_document_passes=10,
                           seed=79,
                           reuse_theta=True,
                           cache_theta=True,
                           scores=[artm.TopTokensScore(name='top_tokens_score', num_tokens=30, class_id='@default_class')],
                           regularizers=[artm.SmoothSparseThetaRegularizer(name='SparseTheta', tau=-0.15)])

    dictionary=artm.Dictionary(name='dictionary')
    dictionary.gather(batch_vectorizer.data_path)

    model_artm.initialize('dictionary')

    dictionary.filter(min_tf=2,  min_df_rate=0.01)

    model_artm.scores.add(artm.SparsityPhiScore(name='sparsity_phi_score',class_id='@labels_class'))
    model_artm.regularizers.add(artm.DecorrelatorPhiRegularizer(name='decorrelator_phi_def',class_ids=['@default_class']))
    model_artm.regularizers.add(artm.DecorrelatorPhiRegularizer(name='decorrelator_phi_lab',class_ids=['@labels_class']))

    model_artm.scores.add(artm.PerplexityScore(name='PerplexityScore',dictionary='dictionary'))
    model_artm.fit_offline(batch_vectorizer=batch_vectorizer, num_collection_passes=15)

    test_transformed = model_artm.transform(batch_vectorizer_test, predict_class_id ='@labels_class').T
    train_transformed = model_artm.transform(batch_vectorizer, predict_class_id ='@labels_class').T

    test_transformed = test_transformed.reset_index().sort_values('index')
    test_transformed = test_transformed.reset_index(drop=True)
    del test_transformed['index']
    test_transformed = test_transformed[sorted(test_transformed.columns)]
    train_transformed = train_transformed.reset_index().sort_values('index')
    del train_transformed['index']
    train_transformed = train_transformed[sorted(train_transformed.columns)]
    train_transformed = train_transformed.reset_index(drop=True)
    artm_transformed = pd.concat([train_transformed, test_transformed], axis=0).reset_index(drop=True)

    return artm_transformed

def train_xgboost(train, y, test, proba):

    """Train and predict using xgboost"""

    clf_xgboost = xgboost.XGBClassifier(n_jobs=-1)
    clf_xgboost = clf_xgboost.fit(train, y)
    if proba == 'probability':

        preds = clf_xgboost.predict_proba(test)

        labels2idx = {label: i for i, label in enumerate(clf_xgboost.classes_)}
        preds = np.array(preds)
        sub = pd.DataFrame()
        for label in clf_xgboost.classes_:
            sub[label] = preds[:, labels2idx[label]]
        preds = sub
    else:
        preds = clf_xgboost.predict(test)

    return preds


def train_mlp(train, y, test, proba):

    """Train neural network"""

    clf_mlp = multilayer_perceptron.MLPClassifier(hidden_layer_sizes=(128,64,),
                                        activation='relu',
                                        solver='adam',
                                        alpha=0.0001,
                                        batch_size=64,
                                        learning_rate='constant',
                                        learning_rate_init=0.001,
                                        power_t=0.5,
                                        max_iter=20,
                                        shuffle=True,
                                        random_state=None,
                                        tol=0.0001,
                                        verbose=False,
                                        warm_start=False,
                                        momentum=0.9,
                                        nesterovs_momentum=True,
                                        early_stopping=True,
                                        validation_fraction=0.1,
                                        beta_1=0.9,
                                        beta_2=0.999, epsilon=1e-08)

    clf_mlp = clf_mlp.fit(train, y)
    if proba == 'probability':

        preds = clf_mlp.predict_proba(test)

        labels2idx = {label: i for i, label in enumerate(clf_mlp.classes_)}
        preds = np.array(preds)
        sub = pd.DataFrame()
        for label in clf_mlp.classes_:
            sub[label] = preds[:, labels2idx[label]]
        preds = sub
    else:
        preds = clf_mlp.predict(test)

    return preds

def train_lr(train, y, test, proba):

    """Train and predict using logistic regression"""

    clf_lr = LogisticRegression(solver="liblinear",
                                multi_class="ovr")
    clf_lr = clf_lr.fit(train, y)
    if proba == 'probability':

        preds = clf_lr.predict_proba(test)

        labels2idx = {label: i for i, label in enumerate(clf_lr.classes_)}
        preds = np.array(preds)
        sub = pd.DataFrame()
        for label in clf_lr.classes_:
            sub[label] = preds[:, labels2idx[label]]
        preds = sub

    else:
        preds = clf_lr.predict(test)

    return preds

def make_cross_val(train, y, clf_custom):

    """Show quality on cross-validation"""

    clf = clf_custom
    cv_scores = []
    folds=3
    from sklearn.cross_validation import KFold
    kf = KFold(len(y), n_folds=folds, shuffle=True, random_state=2016)
    for i, (train_, test_) in enumerate(kf):
        dev_X, val_X = train.ix[train_], train.ix[test_]
        dev_y, val_y = y.ix[train_], y.ix[test_]
        preds = clf(dev_X, dev_y, val_X, 'actual')
        cv_scores.append(accuracy_score(preds, val_y))
    # print ('mean quality of{}'.format(clf), (np.mean(cv_scores)))

def mix_predictions_curtesian(preds_all, list_of_indexes):

    """Finding best combination of predictions"""

    ### make cartesian product
    all_list = []
    for i in combinations_with_replacement(list_of_indexes, 2):
        if i[0] !=i[1]:
            all_list.append(list(i))
    for list_of_indexes in all_list:
        for num_val, i in enumerate(list_of_indexes):
            preds_temp = preds_all[preds_all['clf']==i]
            del preds_temp['clf']
            if num_val == 0:
                preds_first=preds_temp.values
            else:
                preds_next = preds_temp.values
                preds_first+=preds_next
        result = pd.DataFrame(preds_first/len(list_of_indexes))
        today_time = datetime.datetime.now()
        result['clf'] = str(today_time.hour) + '_' + str(today_time.minute) + '_' +  str(today_time.second) + '_' +  str(today_time.microsecond)
        result.columns = preds_all.columns
        preds_all = pd.concat([preds_all, result], axis=0)
    return preds_all

def mix_predictions(preds_all, list_of_indexes):

    """Average blending of appropriate predictions"""

    for num_val, i in enumerate(list_of_indexes):
        preds_temp = preds_all[preds_all['clf']==i]
        del preds_temp['clf']
        if num_val == 0:
            preds_first=preds_temp.values
        else:
            preds_next = preds_temp.values
            preds_first+=preds_next
    result = pd.DataFrame(preds_first/len(list_of_indexes))
    today_time = datetime.datetime.now()
    result['clf'] = str(today_time.hour) + '_' + str(today_time.minute) + '_' +  str(today_time.second) + '_' +  str(today_time.microsecond)
    result.columns = preds_all.columns
    preds_all = pd.concat([preds_all, result], axis=0)
    return preds_all

def make_submission (preds_all, source_test):

    """Takes top-3 probabilities and concates with source data"""

    temp_value = preds_all['clf']
    preds_all= preds_all.loc[:,preds_all.columns !='clf']

    max_3_all =[]
    for i in range(len(preds_all)):
        nums = preds_all.iloc[i].values
        max_3 = heapq.nlargest(3, nums)
        max_3_all.append(max_3)

    preds_all = pd.concat([preds_all.reset_index(drop=True), pd.DataFrame(max_3_all)], axis=1)
    cols = preds_all.iloc[:,:preds_all.shape[1]-3].columns
    target_cols=[0,1,2]

    for i in cols:
        for target_col in target_cols:
            preds_all.loc[preds_all[i]==preds_all[target_col], 'max_value{}'.format(target_col)] = i
    preds_all['clf'] = temp_value.values

    for i in preds_all['clf'].unique():
        submission = pd.concat([preds_all[preds_all['clf']==i].reset_index(drop=True), source_test],axis=1)
        try:
            submission['score_true'] = (submission['max_value0']==submission[target])
            submission['score_true'] = submission['score_true'].astype(int)
            score_true = submission['score_true'].sum()
            print (i)
            print (score_true/len(submission))
        except:
            pass
    submission.to_json('../output/promiss/submisssion{}.json'.format(i))
    return submission

def split_test (path_test):

    """If test shape is higher than  60l script splits into 25k batches"""

    test_to_split = pd.read_excel(path_test)
    num_parts = int(round(test_to_split.shape[0]/25000))
    batch_size = int(test_to_split.shape[0]/num_parts)
    test_to_split.shape[0]/7
    ### нарезка файлов
    max_index=0
    for i in range(int(num_parts)):
        part = test_to_split.iloc[max_index:(max_index+batch_size)]
        max_index = part.index.max()+1
        part.to_excel('input/10_albo_test_{}.xlsx'.format(i), index=False)
        print (max_index)

    all_dfs_input = pd.DataFrame()
    for i in range(int(num_parts)):
        temp = pd.read_excel('input/10_albo_test_{}.xlsx'.format(i))
        all_dfs_input = pd.concat([all_dfs_input, temp],axis=0)

    all_dfs_input.to_excel('temp_input3.xlsx', index=False)

import random

def reduce_train(paths_train):

    """If class count is higher than 3000 script cuts all samples above 3000"""

    pre_train = pd.read_excel(path_train)
    pre_train = pre_train.rename(columns={u'Текст письма': u'Суть обращения', u'Процесс 1' : u'Процесс'})
    pre_train[u'Суть обращения'] = pre_train[u'Категория'].astype(str).fillna(0).replace('nan', '') + ' ' + pre_train[u'Суть обращения'].astype(str)
    pre_train = pre_train[[u'Суть обращения', u'Процесс']]

    cut_df = pd.DataFrame()
    for i in pre_train[u'Процесс'].unique():
        temp = pre_train[pre_train[u'Процесс'] == i]
        if temp.shape[0]>3000:
            temp = temp[:3000]
        else:
            temp = temp
        cut_df = pd.concat([cut_df, temp], axis=0)

    cut_df.to_excel('input/train_albo_v4.xlsx', index=False)

def reduce_train_prob (paths_train):
    all_trains = pd.DataFrame()
    for i in paths_train:
        temp_df = pd.read_excel(i)
        temp_df = temp_df.rename(columns = {u'Категория_CJM':u'Процесс 1', u'Проблематика_Лиза':u'Проблема'})
        temp_df = temp_df[[u'Категория', u'Тема письма', u'Текст письма',u'Процесс 1', u'Проблема']]
        all_trains = pd.concat([all_trains, temp_df],axis=0)


    all_trains[u'Категория'] = all_trains[u'Категория'].apply(lambda x : str(x).replace(u'Стандарт', ''))
    all_trains[u'Суть обращения'] = all_trains[u'Категория'].astype(str)  + ' ' + all_trains[u'Тема письма'].astype(str)+ ' '+ all_trains[u'Текст письма'] .astype(str)
    all_trains['count'] = all_trains[u'Суть обращения'].apply(lambda x : len(x))
    all_trains = all_trains[all_trains['count']>15]
    all_trains[u'Процесс'] = all_trains[u'Процесс 1'] + '_' + all_trains[u'Проблема']
    all_trains = all_trains[[u'Суть обращения', u'Процесс']]
    cut_df = pd.DataFrame()
    for i in all_trains[u'Процесс'].unique():
        temp = all_trains[all_trains[u'Процесс'] == i]
        if temp.shape[0]>1500:
            temp = temp.sample(1500)
        elif temp.shape[0]<50:
            temp = pd.DataFrame()
        else:
            temp = temp
        cut_df = pd.concat([cut_df, temp], axis=0)
#         cut_df.to_excel('input/train_albo_v4_prob.xlsx', index=False)
    return cut_df

def delete_operator(path_sent):
    chat = pd.read_excel(path_sent)
    chat[u'Суть обращения'] = chat[u'Суть обращения'].apply(lambda x : x+'>>>')
    chat[u'Суть обращения'] = chat[u'Суть обращения'].apply(lambda x : x.replace('operator:', '<<<'))
    chat[u'Суть обращения'] = chat[u'Суть обращения'].apply(lambda x : x.replace('client:', '>>>'))
    def replace_operator (col):
        return re.sub('<<<[^>]+>>>', '', col)
    chat[u'Суть обращения'] = chat[u'Суть обращения'].apply(replace_operator)
    chat.to_excel('input/10_chat_v2_sent.xlsx')

In [2]:
%%time
# train = pd.read_csv('../input/toxic/train.csv', nrows=2000)
# test = pd.read_csv('../input/toxic/test.csv', nrows=2000)
train = pd.read_csv('../input/toxic/train.csv')
test = pd.read_csv('../input/toxic/test.csv')
# print (train.shape[0])
# print (test.shape[0])
cols = train.columns
# for i in cols[2:]:
#     print (i, train[i].sum())
sum_of_all = []
for ind in range(train.shape[0]):
    temp = train.iloc[ind]
    sum_of_all.append(temp[cols[2:]].sum())
train['all'] = sum_of_all
cols = train.columns
summing=0
for i in cols[2:]:
#     print (i, train[i].sum())
    if i != 'all':
        summing+=train[i].sum()
hot_df = train[train['all']!=0]
cols = train.columns[2:]
for i in cols:
    test[i]=0
train['train_test'] = 1
test['train_test'] = 0
train_test = pd.concat([train, test])
train_test = train_test.reset_index(drop=True)
text = train_test['comment_text'][0]
# porter_stemmer = PorterStemmer()
# porter_stemmer = stem.porter.PorterStemmer()
stemmer=PorterStemmer()
morph = MorphAnalyzer()
stop = stopwords.words('english')
train_test['comment_text_proc'] = train_test['comment_text'].apply(wrk_words_wt_no)
train_test = train_test.rename(columns={'comment_text_proc':0})


CPU times: user 10min 37s, sys: 1.62 s, total: 10min 39s
Wall time: 10min 38s


In [12]:
# train_test.to_json('../input/toxic/train_test.json')

In [120]:
%%time
train_test=pd.read_json('../../input/toxic/train_test.json')

CPU times: user 3.95 s, sys: 76 ms, total: 4.03 s
Wall time: 4.03 s


In [ ]:
### for func of lang detecting
train_test = train_test.rename(columns={'0':0})

In [122]:
train_test_small = train_test[:1000]

In [87]:
isascii = lambda s: len(s) == len(s.encode())

In [100]:
s.encode()

b'explanation edit make username hardcore metallica fan revert vandalism closure gas vote new york doll fac please remove template talk page since retire '

In [115]:
def is_ascii(s):
    return all(ord(c) < 128 for c in s)

In [126]:
train_test_small['is ascii']  = train_test_small['0'].apply(is_ascii)

/root/nexar/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [98]:
# train_test_small['is ascii'] = train_test_small[0].apply(lambda x : len(x) == len(s.encode())).astype(int)

/root/nexar/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [135]:
non_ascii_str = train_test_small[train_test_small['is ascii']==False]['0'][10]

In [144]:
import unidecode
is_ascii(unidecode.unidecode(non_ascii_str))

True

In [2]:
import pandas as pd
check_ascii = pd.read_csv('../../output/train_test_ascii.csv')

In [3]:
check_ascii

,Unnamed: 0,Unnamed: 0.1,0,all,comment_text,id,identity_hate,insult,obscene,severe_toxic,threat,toxic,train_test,lang,lang_1,lang_2,one_lang,ascii,check is ascii
0,0,0,explanation edit make username hardcore metall...,0,Explanation\nWhy the edits made under my usern...,0000997932d777bf,0,0,0,0,0,0,1,"('en', 'en')",en,en,1,explanation edit make username hardcore metall...,True
1,1,1,aww match background colour seemingly stick th...,0,D'aww! He matches this background colour I'm s...,000103f0d9cfb60f,0,0,0,0,0,0,1,"('en', 'en')",en,en,1,aww match background colour seemingly stick th...,True
2,2,2,fair use rationale image wonju jpg thank uploa...,0,"""\nFair use rationale for Image:Wonju.jpg\n\nT...",0005300084f90edc,0,0,0,0,0,0,1,"('en', 'en')",en,en,1,fair use rationale image wonju jpg thank uploa...,True
3,3,3,however moonlite edit note golden daph optus w...,0,"However, the Moonlite edit noted by golden dap...",003f698d06c9b180,0,0,0,0,0,0,1,"('en', 'en')",en,en,1,however moonlite edit note golden daph optus w...,True
4,4,4,rex mundi create stub rex mundi rex mundi high...,0,Rex Mundi \n\nI've created a stub on Rex Mundi...,02baab63953460ab,0,0,0,0,0,0,1,"('en', 'en')",en,en,1,rex mundi create stub rex mundi rex mundi high...,True
5,5,5,hi redrose apology delay draft propose present...,0,Hi RedRose and apologies for delay. Here is a ...,1a7d7c88372e5668,0,0,0,0,0,0,1,"('en', 'en')",en,en,1,hi redrose apology delay draft propose present...,True
6,6,6,think meet notability guideline thank create p...,0,I think she meets Notability guidelines and th...,173558f5c2a8883b,0,0,0,0,0,0,1,"('en', 'en')",en,en,1,think meet notability guideline thank create p...,True
7,7,7,nice tavern article article like joy wikipedia...,0,"""\n\n Nice tavern article \n\nArticles like th...",17359f940f219872,0,0,0,0,0,0,1,"('en', 'en')",en,en,1,nice tavern article article like joy wikipedia...,True
8,8,8,mfd nomination user wikitrevor sandbox opinion...,0,MfD nomination of User:Wikitrevor/Sandbox\n. Y...,173693e634e1a6cf,0,0,0,0,0,0,1,"('en', 'en')",en,en,1,mfd nomination user wikitrevor sandbox opinion...,True
9,9,9,speak sale speak percentage much hard obtain s...,0,I am not speaking of sales. I am speaking of p...,17375e3fff2d9d1a,0,0,0,0,0,0,1,"('en', 'en')",en,en,1,speak sale speak percentage much hard obtain s...,True


In [18]:
check_spell = pd.read_csv('../../output/train_test_lang_corrected_2.csv')

In [19]:
check_spell.shape

(1000, 21)

In [24]:
check_spell['ascii']==check_spell['corrected']

0      True
1      True
2      True
3      True
4      True
5      True
6      True
7      True
8      True
9      True
10     True
11     True
12     True
13     True
14     True
15     True
16     True
17     True
18     True
19     True
20     True
21     True
22     True
23     True
24     True
25     True
26     True
27     True
28     True
29     True
       ... 
970    True
971    True
972    True
973    True
974    True
975    True
976    True
977    True
978    True
979    True
980    True
981    True
982    True
983    True
984    True
985    True
986    True
987    True
988    True
989    True
990    True
991    True
992    True
993    True
994    True
995    True
996    True
997    True
998    True
999    True
Length: 1000, dtype: bool

In [23]:
pd.options.display.max_colwidth = 400
check_spell

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,0,all,comment_text,id,identity_hate,insult,obscene,...,toxic,train_test,lang,lang_1,lang_2,one_lang,ascii,check is ascii,corrected,check
0,0,0,0,explanation edit make username hardcore metallica fan revert vandalism closure gas vote new york doll fac please remove template talk page since retire,0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0000997932d777bf,0,0,0,...,0,1,"('en', 'en')",en,en,1,explanation edit make username hardcore metallica fan revert vandalism closure gas vote new york doll fac please remove template talk page since retire,True,explanation edit make username hardcore metallica fan revert vandalism closure gas vote new york doll fac please remove template talk page since retire,1
1,1,1,1,aww match background colour seemingly stick thank talk january utc,0,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",000103f0d9cfb60f,0,0,0,...,0,1,"('en', 'en')",en,en,1,aww match background colour seemingly stick thank talk january utc,True,aww match background colour seemingly stick thank talk january utc,1
2,2,2,2,fair use rationale image wonju jpg thank upload image wonju jpg notice image page specify image use fair use explanation rationale use wikipedia article constitute fair use addition boilerplate fair use template must also write image description page specific explanation rationale use image article consistent fair use please go image description page edit include fair use rationale upload fair...,0,"""\nFair use rationale for Image:Wonju.jpg\n\nThanks for uploading Image:Wonju.jpg. I notice the image page specifies that the image is being used under fair use but there is no explanation or rationale as to why its use in Wikipedia articles constitutes fair use. In addition to the boilerplate fair use template, you must also write out on the image description page a specific explanation or ra...",0005300084f90edc,0,0,0,...,0,1,"('en', 'en')",en,en,1,fair use rationale image wonju jpg thank upload image wonju jpg notice image page specify image use fair use explanation rationale use wikipedia article constitute fair use addition boilerplate fair use template must also write image description page specific explanation rationale use image article consistent fair use please go image description page edit include fair use rationale upload fair...,True,fair use rationale image wonju jpg thank upload image wonju jpg notice image page specify image use fair use explanation rationale use wikipedia article constitute fair use addition boilerplate fair use template must also write image description page specific explanation rationale use image article consistent fair use please go image description page edit include fair use rationale upload fair...,1
3,3,3,3,however moonlite edit note golden daph optus wake wikkis funny,0,"However, the Moonlite edit noted by golden daph was me (on optus ...) Wake up wikkis. So funny",003f698d06c9b180,0,0,0,...,0,1,"('en', 'en')",en,en,1,however moonlite edit note golden daph optus wake wikkis funny,True,however moonlite edit note golden daph optus wake wikkis funny,1
4,4,4,4,rex mundi create stub rex mundi rex mundi high school thing know aunt donna bob griese go please add anything might know btw dad panther live princeton,0,"Rex Mundi \n\nI've created a stub on Rex Mundi at Rex Mundi High School. Only thing I know about it is that both my Aunt Donna and Bob Griese went there. Please add anything you might know about it.\n\nBTW, my dad was a Panther; I live in Princeton myself.",02baab63953460ab,0,0,0,...,0,1,"('en', 'en')",en,en,1,rex mundi create stub rex mundi rex mundi high school thing know aunt donna bob griese go please add anything might know btw dad panther

In [22]:
check_spell[check_spell['check']==0]

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,0,all,comment_text,id,identity_hate,insult,obscene,...,toxic,train_test,lang,lang_1,lang_2,one_lang,ascii,check is ascii,corrected,check


In [94]:
s = train_test[0][0]
lambda s: len(s) == len(s.encode())

<function __main__.<lambda>>

In [93]:
isascii

<function __main__.<lambda>>

In [ ]:
def detect_lang(col):
    try:
        shape = int(len(col)/2)
        col1 = col[:shape]
        col2 = col[shape:]
        return TextBlob(col1).detect_language(),TextBlob(col2).detect_language()
    except:
        return 'Unkmown'

In [49]:
train_test = train_test.reset_index(drop=True)

In [42]:
train_test= train_test[160000:165000]
train_test['lang'] =train_test[0].apply(detect_lang)

In [43]:
train_test

,0,all,comment_text,id,identity_hate,insult,obscene,severe_toxic,threat,toxic,train_test,lang
160000,inferior irrelevant change show superior,0,How is it inferior is irrelevant. You've changed it and you cannot show me how it is superior.,8cca653bb4ca1bef,0,0,0,0,0,0,0,"(en, en)"
160001,yang jian claim ancestry yang zhen want extensive list look tomorrow,0,":Yang Jian claimed ancestry from Yang Zhen. Did you want a more extensive list? If so, I can look at it tomorrow.",8cca94ba5179b042,0,0,0,0,0,0,0,"(zh-CN, en)"
160002,name troy tedford troy telford look orginal discharge paper right also orginal purple heart olc man war hero big world war ever historian history degree recieved dec atu,0,"His name was Troy Tedford, not Troy Telford. I am looking at his orginal Discharge papers right now. I also have his orginal purple heart with 2 olc. The man was a war hero in the biggest World Wa...",009b3b15f1ada72f,0,0,0,0,0,0,1,"(en, en)"
160003,agree move back forth bridge accord calculation favour move vote,0,I agree that this should be moved back to Forth Bridge - according to my calculations that's 5 in favour of the move and 2 against any more votes?,0683e21d05624e2d,0,0,0,0,0,0,1,"(en, en)"
160004,obnoxious piece shit homosexual always viscious little fascist,4,You obnoxious piece of shit \n\nWhy are you homosexuals always such viscious little fascists?,4077742c1855d6bd,1,1,1,0,0,1,1,"(en, en)"
160005,yes google search find etymology spanish http www yabiladi com forum read html,0,"Yes after a google search, found this etymology (in Spanish) \n http://www.yabiladi.com/forum/read-75-1744448.html",8ccab5e60961726d,0,0,0,0,0,0,0,"(en, en)"
160006,long disruptively edit dj earworm hey favre fan leave fuck alone admit first edit dj earworm little overload win one unite state pop top song billboard year end hot single suppose list remove fuck...,0,""" \n\n == I AM NO LONGER DISRUPTIVELY EDITING DJ EARWORM!!! == \n\n HEY, FAVRE1FAN93!!!! LEAVE ME THE FUCK ALONE!!!! I'LL ADMIT THAT SOME OF MY FIRST EDITS TO DJ EARWORM WERE A LITTLE OVERLOADING ...",8ccb0713294f2fab,0,0,0,0,0,0,0,"(en, en)"
160007,*the late example grawp stupidity well intentioned editor blank page though make impossible move vandalize page back appropriate location,0,"== 0.999... == \n\n *The latest example of Grawp stupidity. A well-intentioned editor blanked this page, though, making it impossible to move the vandalized page back to its appropriate location.",8ccb3c0445f795f8,0,0,0,0,0,0,0,"(en, en)"
160008,thank clevenland joe get look familiar feel like tip tongue gon na bother recall july utc,0,"Thanks. Both Clevenland's and Joe's get-ups look so familiar, and I feel like it's on the tip of my tongue... This is gonna bother me until I recall. 4 July 2005 23:15 (UTC)",8ccb5903d9016723,0,0,0,0,0,0,0,"(en, en)"
160009,afl hi aussie rule great sport world least rest world realise pretty hard claim team represent australia aussie rule fantasy football imho ymmv dncwa etc etc regard,0,"AFL \n\n Hi, Aussie rules is the greatest sport in the world, but until at least some of the rest of the world realises that, it's pretty hard to claim that we have a team that represents Australi...",8ccbcc1334097a23,0,0,0,0,0,0,0,"(en, en)"


In [10]:
train_test_lang = pd.read_csv('../../output/train_test_lang.csv')

In [11]:
train_test_lang

,Unnamed: 0,0,all,comment_text,id,identity_hate,insult,obscene,severe_toxic,threat,toxic,train_test,lang
0,0,explanation edit make username hardcore metallica fan revert vandalism closure gas vote new york doll fac please remove template talk page since retire,0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remo...",0000997932d777bf,0,0,0,0,0,0,1,"('en', 'en')"
1,1,aww match background colour seemingly stick thank talk january utc,0,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",000103f0d9cfb60f,0,0,0,0,0,0,1,"('en', 'en')"
2,2,fair use rationale image wonju jpg thank upload image wonju jpg notice image page specify image use fair use explanation rationale use wikipedia article constitute fair use addition boilerplate fa...,0,"""\nFair use rationale for Image:Wonju.jpg\n\nThanks for uploading Image:Wonju.jpg. I notice the image page specifies that the image is being used under fair use but there is no explanation or rati...",0005300084f90edc,0,0,0,0,0,0,1,"('en', 'en')"
3,3,however moonlite edit note golden daph optus wake wikkis funny,0,"However, the Moonlite edit noted by golden daph was me (on optus ...) Wake up wikkis. So funny",003f698d06c9b180,0,0,0,0,0,0,1,"('en', 'en')"
4,4,rex mundi create stub rex mundi rex mundi high school thing know aunt donna bob griese go please add anything might know btw dad panther live princeton,0,Rex Mundi \n\nI've created a stub on Rex Mundi at Rex Mundi High School. Only thing I know about it is that both my Aunt Donna and Bob Griese went there. Please add anything you might know about...,02baab63953460ab,0,0,0,0,0,0,1,"('en', 'en')"
5,5,hi redrose apology delay draft propose present new section page volume ultimate allocation cover report allocation british railway steam diesel & electric locomotive br stock additional informatio...,0,Hi RedRose and apologies for delay. Here is a draft of what I propose to present on the 'new section' pages.\nThe 6 volumes of Ultimate Allocations cover the 'reported' allocations of all British ...,1a7d7c88372e5668,0,0,0,0,0,0,1,"('en', 'en')"
6,6,think meet notability guideline thank create page big icon pageant drag community drag gay community general death cover several mainstream medium outlet,0,I think she meets Notability guidelines and thanks for creating the page for her. She was a big icon in the pageant drag community and in the drag/gay community in general. Her death has been cove...,173558f5c2a8883b,0,0,0,0,0,0,1,"('en', 'en')"
7,7,nice tavern article article like joy wikipedia keep good work thank appreciate plan eat first time two week seek grace,0,"""\n\n Nice tavern article \n\nArticles like this are the joy of Wikipedia. Keep up the good work. \nThanks. I appreciate it. I plan to eat there for the first time in two weeks. Seek his grace """,17359f940f219872,0,0,0,0,0,0,1,"('en', 'en')"
8,8,mfd nomination user wikitrevor sandbox opinion matter welcome please participate discussion add comment wikipedia miscellany deletion user wikitrevor sandbox please sure sign comment four tilde ~~...,0,MfD nomination of User:Wikitrevor/Sandbox\n. Your opinions on the matter are welcome; please participate in the discussion by adding your comments at Wikipedia:Miscellany for deletion/User:Wikitre...,173693e634e1a6cf,0,0,0,0,0,0,1,"('en', 'en')"
9,9,speak sale speak percentage much hard obtain sale percentage say half people p resistance fall man gear war sell million copy xbox sell million copy gear war little fact speak,0,"I am not speaking of sales. I am speaking of percentage, much harder to obtain than sales, and that percentage says more than half of the people who own PS3's own Resistance: Fall of Man. Gears of...",17375e3fff2d9d1a,0,0,0,0,0,0,1,"('en', 'en')"


In [44]:
train_test['lang'] = train_test['lang'].astype(str)

In [45]:
not_en_df = train_test[train_test['lang']!="""('en', 'en')"""]

In [34]:
('en', 'en')

('en', 'en')

In [46]:
not_en_df[not_en_df['all']!=0]

,0,all,comment_text,id,identity_hate,insult,obscene,severe_toxic,threat,toxic,train_test,lang
160337,let help sock puppet,1,Let me help you out here \n\nI'm a sock puppet too! 70.3.5.246,409337267a5540e7,0,0,0,0,0,1,1,"('en', 'sv')"
161281,hello turd firśt aṃ ģoinģ ţo ţie ŷou uṗ keeṗ ŷou çonśçiouś durinģ ţhe folloŵinģ ṗroçeśś ţhrough ţhe uśe aṃṃonia nexţ ŵill ṃuţilaţe ŷour ģeniţalś forçe ŷou ţo eaţ ţheṃ ŵill çuţ biţś ŷour śkin ṃake ...,4,"Hello, you turd. \n\nFirśt, I aṃ ģoinģ ţo ţie ŷou uṗ and keeṗ ŷou çonśçiouś durinģ ţhe folloŵinģ ṗroçeśś ţhrough ţhe uśe of aṃṃonia. Nexţ, I ŵill ṃuţilaţe ŷour ģeniţalś and forçe ŷou ţo eaţ ţheṃ. ...",40c5011b970a3529,0,1,1,0,1,1,1,"('ro', 'tr')"
162403,dear omni loose stop edit crap die,4,dear omni looser stop editing my crap then die,41123edb3bc7bf76,0,1,1,0,1,1,1,"('en', 'af')"
163158,vandalism douche faggotry tolerate website,1,"Vandalism, douching, and faggotry are not tolerated on this website.",4144ab3662323627,0,0,1,0,0,0,1,"('en', 'mg')"
163603,trouted trouted reason hole,3,Trouted \n\nYou have been trouted for: YOUR REASON HERE\nAss Hole,4158f0f914760b1f,0,1,1,0,0,1,1,"('fy', 'en')"


In [50]:
collection = train_test[0]

In [ ]:
# try:
# %%time
bow_df= make_bow(collection)

train_test = train_test.reset_index(drop=True)
train_index = train_test[train_test['train_test']==1].index
test_index = train_test[train_test['train_test']==0].index

In [82]:
w2v, model = make_w2v(collection)

In [85]:
model.most_similar('suck')

[('notrhbysouthbanof', 0.7206345200538635),
 ('chodes', 0.6865939497947693),
 ('oshwah', 0.6823679804801941),
 ('ullmann', 0.6789976358413696),
 ('ballsarod', 0.6756810545921326),
 ('fuc*ers', 0.6618494391441345),
 ('epbr', 0.6616906523704529),
 ('bestfrozen', 0.6598241925239563),
 ('sucksucku', 0.6587342619895935),
 ('di*k', 0.6584660410881042)]

In [ ]:
# %%time
###подготовим датасеты
# bow_df = make_bow(collection)
# w2v_mean_df= make_w2v_mean(collection)
w2v_tfidf_df =  make_w2v_tfidf(collection)

artm_df = make_artm(collection)
print (bow_df.shape[0])
# print w2v_mean_df.shape[0]
print (w2v_tfidf_df.shape[0])
print (artm_df.shape[0])

list_clfs = [train_xgboost, train_lr]
data_sources = [bow_df, w2v_tfidf_df, artm_df]

In [9]:
send_to_telegram('препроцессинг сделан')

In [ ]:
    # list_clfs = [train_xgboost]
    # data_sources = [bow_df]

    # bow_df = pd.DataFrame(bow_df.toarray())

    # %%time
try:    
    preds_all=pd.DataFrame()
    targets = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
    for target in targets:
        print (target)
        df_y = train_test.iloc[train_index][target]
    #     artm_df = make_artm(collection)
    #     list_clfs = [train_lr]
    #     data_sources = [l]

        for num_source, source in enumerate(data_sources):
            print (num_source)
            train = source.iloc[train_index]
            y = train_test.iloc[train_index][target]
            test = source.iloc[test_index]
            for num_clf, i in enumerate(list_clfs):
                if (num_source == 0) & (num_clf == 1):
                    print ('пропускаем')
                    pass
                elif (num_source == 1) & (num_clf == 0):
                    print ('пропускаем')
                    pass
                elif (num_source == 2) & (num_clf == 0):
                    print ('пропускаем')
                    pass
                else:
                    print (i)
                    preds = i(train, y, test, 'probability')
                    preds = pd.DataFrame(preds)
                    preds['clf']=str(num_source) + ' ' + str(num_clf)
                    preds_all = pd.concat([preds_all, preds], axis=1)
    send_to_telegram('все сделано')
    # preds_all = mix_predictions(preds_all, ['0 0', '1 1', '2 1'])
except:
    send_to_telegram('ошибка')
    

In [68]:
# preds_all

In [14]:
sub = pd.read_csv('sample_submission.csv')
submission = pd.concat([sub[['id']], preds_all[1]],axis=1)

In [15]:
submission.columns = ('id', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate')

In [16]:
submission.to_csv('sub_xgb_20170402.csv', index=False)

In [17]:
!zip -r sub_xgb_20170402.csv.zip sub_xgb_20170402.csv

  adding: sub_xgb_20170402.csv (deflated 72%)


In [2]:
import os
from functools import reduce
sub_dir = '../input/toxic/subs/'

files=[]
for i in os.listdir(sub_dir):
    files.append(sub_dir+ i)

preds = [pd.read_csv(f, index_col='id') for f in files]
pred = reduce(lambda a, b: a + b, preds) / len(preds)
pred = pred.reset_index()
pred.to_csv('avg_20180204_v4.csv', index=False)

In [3]:
import pandas as pd

In [4]:
pd.read_json('preds_all.json')

""


In [3]:
pred.columns

Index(['id', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')

In [4]:
!zip -r avg_20180204_v4.csv.zip avg_20180204_v4.csv

  adding: avg_20180204_v4.csv (deflated 53%)


In [5]:
import telebot

In [15]:
bot = telegram.Bot(token='451841380:AAFLpx5Tm8f-3VEGx2QEPEz3EAeQXwnjG0U')

In [27]:
API_TOKEN = '451841380:AAFLpx5Tm8f-3VEGx2QEPEz3EAeQXwnjG0U'

bot = telebot.TeleBot(API_TOKEN)

In [21]:
chat_id = -1001289775579

In [32]:
import telegram, telegram.ext

tb = telegram.ext.Updater(API_TOKEN)
def a(bot, update):
    bot.sendMessage(update.message.chat_id, "responding to /start cmd")

tb.dispatcher.add_handler(telegram.ext.CommandHandler("start", '@like'))
tb.start_polling()

In [25]:
mes_id= bot.send_message(chat_id, 'text').message_id

In [26]:
mes_id

20

In [13]:
bot.send_message?

In [62]:
# pd.read_csv('../input/toxic/subs/sub_xgb_20170402.csv')

In [63]:
# pd.read_csv('avg_20180204_v2.csv')

In [67]:
# sub[:2]

In [66]:
# pred.reset_index()[:2]

In [65]:
# submission[:2]

In [64]:
# preds_all[1]

In [136]:
preds_all['clf'].unique()

array(['0 0'], dtype=object)

In [142]:
test_size = sub.shape[0]

In [137]:
preds_all.shape

(918984, 3)

In [141]:
num_parts = preds_all.shape[0]/sub.shape[0]

In [144]:
preds_all['check'] = 0

In [150]:
preds_all.index.max()

153163

In [155]:
new_sub = pd.DataFrame()
for _ in range(int(num_parts)):
    part = preds_all[:test_size]
    preds_all = preds_all[test_size:]
    new_sub = pd.concat([new_sub, part],axis=1)
    
    

In [156]:
new_sub

,0,1,clf,check,0,1,clf,check,0,1,...,clf,check,0,1,clf,check,0,1,clf,check


In [95]:
# preds_all = mix_predictions(preds_all, ['0 0', '1 0'])
#
source_test=read_data_test(test_path)###for general test
# source_test = y_true.reset_index(drop=True)#for validation

sub = make_submission(preds_all, source_test)
sub = sub[['max_value0']]
#parsed_date_money = pd.read_excel('/root/projects/output/promiss/parsed_date_money.xlsx')
parsed_date_money = pd.read_json('/root/projects/output/promiss/parsed_date_money.json')
parsed_date_money = pd.concat([parsed_date_money, sub], axis=1)
#parsed_date_money.to_excel('/root/projects/output/promiss/parsed_date_money_classification.xlsx')
parsed_date_money.to_json('/root/projects/output/promiss/parsed_date_money_classification.json')

NameError: name 'read_data_test' is not defined

In [7]:
temp_data = pd.read_csv('../input/toxic/train_test_temp.csv')

In [158]:
pd.options.display.max_colwidth = 200
pd.options.display.max_rows = 200

In [15]:
temp_data[['comment_text', '0']]

,comment_text,0
0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remo...",explanation edit make username hardcore metallica fan revert vandalism closure gas vote new york doll fac please remove template talk page since retire
1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",aww match background colour seemingly stick thank talk january utc
2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about...",hey man really try edit war guy constantly remove relevant information talk edit instead talk page seem care format actual info
3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tid...",make real suggestion improvement wonder section statistic late subsection type accident think reference may need tidy exact format ie date format etc late one else first preference format style re...
4,"You, sir, are my hero. Any chance you remember what page that's on?",sir hero chance remember page
5,"""\n\nCongratulations from me as well, use the tools well. · talk """,congratulation well use tool well · talk
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,cocksucker piss around work
7,"Your vandalism to the Matt Shirvington article has been reverted. Please don't do it again, or you will be banned.",vandalism matt shirvington article revert please ban
8,"Sorry if the word 'nonsense' was offensive to you. Anyway, I'm not intending to write anything in the article(wow they would jump on me for vandalism), I'm merely requesting that it be more encycl...",sorry word nonsense offensive anyway intend write anything article wow would jump vandalism merely request encyclopedic one use school reference selective breed page almost stub point animal breed...
9,alignment on this subject and which are contrary to those of DuLithgow,alignment subject contrary dulithgow


In [164]:
lang_df = pd.read_csv('../../output/train_test_lang.csv')

In [177]:
def split_col_1 (col):
    try:
        col1 = col.split("', '")[0].split("('")[1]
        return col1
    except:
        return 'unknown'

lang_df['lang_1'] = lang_df['lang'].apply(split_col_1)

def split_col_2 (col):
    try:
        col1 = col.split("', '")[1].split("')")[0]
        return col1
    except:
        return 'unknown'

lang_df['lang_2'] = lang_df['lang'].apply(split_col_2)
lang_df['one_lang'] = (lang_df['lang_1']==lang_df['lang_2']).astype(int)
# lang_df[lang_df['one_lang']==1].shape[0]/lang_df.shape[0]

one_lang = lang_df[lang_df['one_lang']==1]

# one_lang.pivot_table('id', ['train_test', 'lang_1'], aggfunc='count')

one_lang_no_eng = one_lang[one_lang['lang_1'] !='en']

In [212]:
one_lang_no_eng_train = one_lang_no_eng[one_lang_no_eng['train_test']==1]

In [214]:
one_lang_no_eng_train_toxic = one_lang_no_eng_train[one_lang_no_eng_train['all']>=1]

In [216]:
one_lang_no_eng_train_toxic = one_lang_no_eng_train_toxic[one_lang_no_eng_train_toxic['lang_1'] != 'unknown']

In [25]:
# one_lang_no_eng_train[one_lang_no_eng_train['lang_1']!='unknown']

In [ ]:
one_lang_no_eng = one_lang[one_lang['lang_1'] !='en']

In [226]:
no_one_lang = lang_df[lang_df['one_lang']==0]

In [ ]:
def split_col_1 (col):
    try:
        col1 = col.split("', '")[0]
    except:
        return 'unknown'

In [146]:
lang_df.shape

(312735, 13)

In [155]:
lang_ru = lang_df[lang_df['lang']=="""('ru', 'ru')"""]

In [162]:
pd.options.display.max_colwidth=400

In [163]:
lang_ru[lang_ru['all']>=0]

,Unnamed: 0,0,all,comment_text,id,identity_hate,insult,obscene,severe_toxic,threat,toxic,train_test,lang
21201,21201,memorial victim holodomor ukraine national museum mazepy street kyiv http primetour ua en excursion museum natsionalnyiy muzeymemorial pamyati zhertv golodomorov v ukraine html,0,"Memorial to the Victims of Holodomor in Ukraine National Museum - is it the same? 15-a, I. Mazepy Street, Kyiv\n\nhttp://primetour.ua/en/excursions/museum/Natsionalnyiy-muzeyMemorial-pamyati-zhertv-golodomorov-v-Ukraine-.html",7c835619255a00cc,0,0,0,0,0,0,1,"('ru', 'ru')"
58226,58226,petr v|petr v,0,Petr V|Petr V]] 17:47,283ec6cdb61a7f86,0,0,0,0,0,0,1,"('ru', 'ru')"
66658,66658,радченко иса мы дирот гаы анал сех слава,0,Радченко ис мы дирты гаы анал сех славе!!,00b568d3b0f61a37,0,0,0,0,0,0,0,"('ru', 'ru')"
69749,69749,butu pytatsya pisati po ruskom u menia ruskii ocheni plohoi tak chto izvini esli dumaesi chto daty nepravilynie nujno izmeniti v statie v dvuh mestah v nachiale stati v categorii pod statyoi nujno takje izmeniti v list mathematician na vsiakii sluchai esli moi bot ne doidiot tam esli ne hocheshi tam kelvina nujno ubrati ego list mathematician takje matematicheskoi categorii togda moi bot ne po...,0,": Butu pytatsya pisati po ruskom, no u menia ruskii ocheni plohoi, tak chto izvini. \n\n : Esli dumaesi chto daty nepravilynie, nujno izmeniti v statie v dvuh mestah, v nachiale stati i v categorii pod statyoi. Nujno takje izmeniti v List of mathematicians, na vsiakii sluchai esli moi bot ne doidiot tam. \n\n : Esli ne hocheshi tam Kelvina, nujno ubrati ego is List of mathematicians, i takje ...",0580e47fd1ae3d1a,0,0,0,0,0,0,0,"('ru', 'ru')"
77111,77111,speak russian пожалуйста не_постоянно менять альбом что художник сделать в раздел ссылка он не_принадлежать там если вы найти источник отдельный от источник добавить они в статья,0,"== DO you speak Russian? == \n\n пожалуйста, не постоянно менять альбомов, что художник сделал в разделе ссылок. Он не принадлежит там, если вы найдете источники отдельный от источника добавить их в статью.",10686472ac5ce742,0,0,0,0,0,0,0,"('ru', 'ru')"
78175,78175,это мой страница обсуждение,0,Это моя страница обсуждения.,1213f8a496bb923c,0,0,0,0,0,0,0,"('ru', 'ru')"
80956,80956,viktor ivanovich kosichkin виктор иван косичкина,0,Viktor Ivanovich Kosichkin Виктор Иванович Косичкин,165e72f6c02b7316,0,0,0,0,0,0,0,"('ru', 'ru')"
81690,81690,я не_помнить такой месяц чтобы англоязычный спам с всякий мета объявление не_вылезти бы на русскоязычный раздел повторять на форум а тут я обращаться непосредственно к носитель язык ну ладный он найти способ сбежать в куст но вы то что с тот не_понимать да ещё пинговать мой ник обсудить этот трагений с ярослав коли уж так весь плохо я не_вплетать что хотеть я уже понять,0,"::: Я не помню такого месяца, чтобы англоязычный спам со всякими мета-объявлениями не вылез бы на русскоязычного раздела. Повторяю: НА ФОРУМЫ. А тут я обращаюсь непосредственно к носителю языка. Ну ладно- он нашел способ сбежать в кусты, но вам то что с того, не понимаю? Да еще пинговать мой ник. Обсудите эту трагению с Ярославом, коли уж так все плохо, меня не вплетайте. Что хотел - я уже понял.",17733d6971383b55,0,0,0,0,0,0,0,"('ru', 'ru')"
82221,82221,world without nazism opinion http worldwithoutnazism org article tegi zayavleniya_liderov_dvigeniya zayavlenie_prezidenta_mpd_mir_bez_natsizma_borisa_shpigelya_v_svyazi_s_vyskazyvaniem_premer ministra_ukrainy_yatsenyuka_v_efire_federalnogo_nemetskogo_telekanala_ard say yatsenyuk act hitler advocate,0,""" \n\n World Without Nazism opinion http://worldwithoutnazism.org/articles/tegi/zayavleniya_liderov_dvigeniya/zayavlenie_prezidenta_mpd_mir_bez_natsizma_borisa_shpigelya_v_svyazi_s_vyskazyvaniem_premer-ministra_ukrainy_yatsenyuka_v_efire_federalnogo_nemetskogo_telekanala_ard/ They say Yatsenyuk acts as Hitler's """"advocate"""" """,1837e6fceb3cc74a,0,0,0,0,0,0,0,"('ru', 'ru')"
82796,82796,search google saw newspaper magazine story therefore delete a

In [151]:
lang_df[:2]

,Unnamed: 0,0,all,comment_text,id,identity_hate,insult,obscene,severe_toxic,threat,toxic,train_test,lang
0,0,explanation edit make username hardcore metallica fan revert vandalism closure gas vote new york doll fac please remove template talk page since retire,0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remo...",0000997932d777bf,0,0,0,0,0,0,1,"('en', 'en')"
1,1,aww match background colour seemingly stick thank talk january utc,0,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",000103f0d9cfb60f,0,0,0,0,0,0,1,"('en', 'en')"
